In [545]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import Imputer, LabelEncoder, OneHotEncoder

In [542]:
data = pd.read_csv('train.csv')

In [543]:
data.head()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
0,189123628,10.5,834,56.0,0.0000,64.0,11/1/2019 00:20,11/1/2019 00:34,6.86252,79.8993,6.90330,79.8783,270.32,correct
1,189125358,10.5,791,47.0,0.0000,134.0,11/1/2019 00:56,11/1/2019 01:09,6.88589,79.8984,6.91373,79.8923,197.85,correct
2,189125719,10.5,1087,80.0,0.0000,61.0,11/1/2019 01:08,11/1/2019 01:26,6.90839,79.8651,6.93669,79.9146,301.64,correct
3,189127273,10.5,598,271.0,15.6638,68.0,11/1/2019 02:27,11/1/2019 02:37,6.92570,79.8895,6.92748,79.8971,82.30,correct
4,189128020,NaN,1020,NaN,NaN,NaN,11/1/2019 03:34,11/1/2019 03:51,6.87441,79.8615,6.84478,79.9290,358.39,correct


In [544]:
data["pickup_time"] = pd.to_datetime(data.pickup_time)
data["drop_time"] = pd.to_datetime(data.drop_time)
data["pickup_year"] = data.pickup_time.dt.year
data["pickup_month"] = data.pickup_time.dt.month
data["pickup_date"] = data.pickup_time.dt.day
data["pickup_hour"] = data.pickup_time.dt.hour
data["pickup_minute"] = data.pickup_time.dt.minute
data["drop_year"] = data.drop_time.dt.year
data["drop_month"] = data.drop_time.dt.month
data["drop_date"] = data.drop_time.dt.day
data["drop_hour"] = data.drop_time.dt.hour
data["drop_minute"] = data.drop_time.dt.minute
del data['pickup_time']
del data['drop_time']

In [532]:
data = data.astype({"duration": int})
data.info()
imputer = SimpleImputer(missing_values=np.nan,strategy='mean')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17176 entries, 0 to 17175
Data columns (total 22 columns):
tripid                       17176 non-null int64
additional_fare              16974 non-null float64
duration                     17176 non-null int32
meter_waiting                16974 non-null float64
meter_waiting_fare           16974 non-null float64
meter_waiting_till_pickup    16974 non-null float64
pick_lat                     17176 non-null float64
pick_lon                     17176 non-null float64
drop_lat                     17176 non-null float64
drop_lon                     17176 non-null float64
fare                         17039 non-null float64
label                        17176 non-null object
pickup_year                  17176 non-null int64
pickup_month                 17176 non-null int64
pickup_date                  17176 non-null int64
pickup_hour                  17176 non-null int64
pickup_minute                17176 non-null int64
drop_year             

In [533]:
data_copy = data


In [535]:
data_na = data.dropna()
del(data_copy['fare'])

In [536]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17176 entries, 0 to 17175
Data columns (total 21 columns):
tripid                       17176 non-null int64
additional_fare              16974 non-null float64
duration                     17176 non-null int32
meter_waiting                16974 non-null float64
meter_waiting_fare           16974 non-null float64
meter_waiting_till_pickup    16974 non-null float64
pick_lat                     17176 non-null float64
pick_lon                     17176 non-null float64
drop_lat                     17176 non-null float64
drop_lon                     17176 non-null float64
label                        17176 non-null object
pickup_year                  17176 non-null int64
pickup_month                 17176 non-null int64
pickup_date                  17176 non-null int64
pickup_hour                  17176 non-null int64
pickup_minute                17176 non-null int64
drop_year                    17176 non-null int64
drop_month              

In [275]:
data_na.shape

(16974, 21)

In [276]:
corr = data_na.corr()

In [277]:
corr

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,pickup_year,pickup_month,pickup_date,pickup_hour,pickup_minute,drop_year,drop_month,drop_date,drop_hour,drop_minute
tripid,1.000000,0.019145,-0.019747,-0.016882,-0.017301,-0.017864,0.029046,0.030015,0.006364,0.030386,0.827282,-0.786633,0.347475,0.023465,-0.002339,0.827353,-0.786724,0.347261,0.025601,-0.008565
additional_fare,0.019145,1.000000,-0.000772,0.197605,0.199604,0.002355,0.009124,0.000621,0.004644,0.001539,0.019370,-0.018938,0.005834,-0.005040,-0.000711,0.019361,-0.018928,0.005844,-0.004698,0.000058
duration,-0.019747,-0.000772,1.000000,0.969787,0.968666,0.040578,-0.027079,-0.025593,-0.015969,-0.013605,-0.015004,0.013198,0.009517,-0.005989,0.001980,-0.014548,0.013424,0.007477,-0.005467,-0.001224
meter_waiting,-0.016882,0.197605,0.969787,1.000000,0.999474,0.030447,-0.016171,-0.014211,-0.010117,-0.007905,-0.010860,0.009206,0.005582,-0.009129,0.002257,-0.010453,0.009473,0.003651,-0.015307,-0.000640
meter_waiting_fare,-0.017301,0.199604,0.968666,0.999474,1.000000,0.029908,-0.015014,-0.012483,-0.009430,-0.006805,-0.010971,0.009290,0.005016,-0.008487,0.002142,-0.010562,0.009555,0.003052,-0.014796,-0.000411
meter_waiting_till_pickup,-0.017864,0.002355,0.040578,0.030447,0.029908,1.000000,0.011393,0.016509,0.006949,0.011065,-0.010902,0.009582,-0.003970,-0.007333,0.013554,-0.010903,0.009587,-0.003527,-0.009210,-0.006264
pick_lat,0.029046,0.009124,-0.027079,-0.016171,-0.015014,0.011393,1.000000,0.519974,0.558316,0.336866,0.014303,-0.012448,0.022115,0.013288,0.002752,0.014389,-0.012551,0.021921,0.012964,0.002192
pick_lon,0.030015,0.000621,-0.025593,-0.014211,-0.012483,0.016509,0.519974,1.000000,0.304984,0.651816,0.021575,-0.020527,0.020443,0.030308,0.006383,0.021521,-0.020480,0.020460,0.028517,-0.005733
drop_lat,0.006364,0.004644,-0.015969,-0.010117,-0.009430,0.006949,0.558316,0.304984,1.000000,-0.420545,0.003250,-0.002921,0.005203,0.001208,0.004126,0.003227,-0.002901,0.005154,0.000668,0.003852
drop_lon,0.030386,0.001539,-0.013605,-0.007905,-0.006805,0.011065,0.336866,0.651816,-0.420545,1.000000,0.019850,-0.018476,0.020197,0.038346,0.001152,0.019834,-0.018468,0.020282,0.035807,-0.008613


In [278]:
corr['meter_waiting'][abs(corr['meter_waiting'])>0.1]

additional_fare       0.197605
duration              0.969787
meter_waiting         1.000000
meter_waiting_fare    0.999474
Name: meter_waiting, dtype: float64

In [443]:
y1 = data_na.meter_waiting
features = ['duration','pick_lat','pick_lon','drop_lat','drop_lon','pickup_hour','pickup_month']
x1 = data_na[features]

In [444]:
x1.shape, y1.shape

((16974, 7), (16974,))

In [450]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
#linReg1 = MLPRegressor(hidden_layer_sizes=(20,20,10,5,),activation='relu',max_iter=200)
linReg1 = RandomForestRegressor()
linReg1.fit(x1,y1)

c:\users\isuru\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [451]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17176 entries, 0 to 17175
Data columns (total 21 columns):
tripid                       17176 non-null int64
additional_fare              16974 non-null float64
duration                     17176 non-null float64
meter_waiting                16974 non-null float64
meter_waiting_fare           16974 non-null float64
meter_waiting_till_pickup    16974 non-null float64
pick_lat                     17176 non-null float64
pick_lon                     17176 non-null float64
drop_lat                     17176 non-null float64
drop_lon                     17176 non-null float64
label                        17176 non-null object
pickup_year                  17176 non-null int64
pickup_month                 17176 non-null int64
pickup_date                  17176 non-null int64
pickup_hour                  17176 non-null int64
pickup_minute                17176 non-null int64
drop_year                    17176 non-null int64
drop_month            

In [452]:
x = data[features]
list1 = linReg1.predict(x)

In [473]:
list1[4]

88.4

In [370]:
data.head()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,...,pickup_year,pickup_month,pickup_date,pickup_hour,pickup_minute,drop_year,drop_month,drop_date,drop_hour,drop_minute
0,189123628,10.5,834.0,56.0,0.0000,64.0,6.86252,79.8993,6.90330,79.8783,...,2019,11,1,0,20,2019,11,1,0,34
1,189125358,10.5,791.0,47.0,0.0000,134.0,6.88589,79.8984,6.91373,79.8923,...,2019,11,1,0,56,2019,11,1,1,9
2,189125719,10.5,1087.0,80.0,0.0000,61.0,6.90839,79.8651,6.93669,79.9146,...,2019,11,1,1,8,2019,11,1,1,26
3,189127273,10.5,598.0,271.0,15.6638,68.0,6.92570,79.8895,6.92748,79.8971,...,2019,11,1,2,27,2019,11,1,2,37
4,189128020,NaN,1020.0,NaN,NaN,NaN,6.87441,79.8615,6.84478,79.9290,...,2019,11,1,3,34,2019,11,1,3,51


In [470]:
str(data.meter_waiting[4]) == "nan"

True

In [471]:
for i in range(0,len(list1)):
    if(str(data.meter_waiting[i])=="nan"):
        data.meter_waiting[i] = list1[i]

c:\users\isuru\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [472]:
data.head()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,...,pickup_year,pickup_month,pickup_date,pickup_hour,pickup_minute,drop_year,drop_month,drop_date,drop_hour,drop_minute
0,189123628,10.5,834.0,56.0,0.0000,64.0,6.86252,79.8993,6.90330,79.8783,...,2019,11,1,0,20,2019,11,1,0,34
1,189125358,10.5,791.0,47.0,0.0000,134.0,6.88589,79.8984,6.91373,79.8923,...,2019,11,1,0,56,2019,11,1,1,9
2,189125719,10.5,1087.0,80.0,0.0000,61.0,6.90839,79.8651,6.93669,79.9146,...,2019,11,1,1,8,2019,11,1,1,26
3,189127273,10.5,598.0,271.0,15.6638,68.0,6.92570,79.8895,6.92748,79.8971,...,2019,11,1,2,27,2019,11,1,2,37
4,189128020,NaN,1020.0,88.4,NaN,NaN,6.87441,79.8615,6.84478,79.9290,...,2019,11,1,3,34,2019,11,1,3,51


In [475]:
y2 = data_na.meter_waiting_fare
features = ['duration','meter_waiting']
x2 = data_na[features]

In [478]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
#linReg1 = MLPRegressor(hidden_layer_sizes=(20,20,10,5,),activation='relu',max_iter=200)
linReg1 = RandomForestRegressor()
linReg1.fit(x2,y2)

c:\users\isuru\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [479]:
x = data[features]
list1 = linReg1.predict(x)

In [480]:
len(list1)

17176

In [484]:
list1[3]

11.00337

In [487]:
for i in range(0,len(list1)):
    if(str(data.meter_waiting_fare[i])=="nan"):
        data.meter_waiting_fare[i] = list1[i]

c:\users\isuru\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [489]:
data.head()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,...,pickup_year,pickup_month,pickup_date,pickup_hour,pickup_minute,drop_year,drop_month,drop_date,drop_hour,drop_minute
0,189123628,10.5,834.0,56.0,0.00000,64.0,6.86252,79.8993,6.90330,79.8783,...,2019,11,1,0,20,2019,11,1,0,34
1,189125358,10.5,791.0,47.0,0.00000,134.0,6.88589,79.8984,6.91373,79.8923,...,2019,11,1,0,56,2019,11,1,1,9
2,189125719,10.5,1087.0,80.0,0.00000,61.0,6.90839,79.8651,6.93669,79.9146,...,2019,11,1,1,8,2019,11,1,1,26
3,189127273,10.5,598.0,271.0,15.66380,68.0,6.92570,79.8895,6.92748,79.8971,...,2019,11,1,2,27,2019,11,1,2,37
4,189128020,NaN,1020.0,88.4,0.51304,NaN,6.87441,79.8615,6.84478,79.9290,...,2019,11,1,3,34,2019,11,1,3,51


In [491]:
y3 = data_na.meter_waiting_till_pickup
features = ["pick_lat", "pick_lon","pickup_hour"]
x3 = data_na[features]

In [492]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
#linReg1 = MLPRegressor(hidden_layer_sizes=(20,20,10,5,),activation='relu',max_iter=200)
linReg1 = RandomForestRegressor()
linReg1.fit(x3,y3)

c:\users\isuru\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [493]:
x = data[features]
list1 = linReg1.predict(x)

In [497]:
list1[4]

100.3

In [498]:
for i in range(0,len(data.additional_fare)):
    if(str(data.additional_fare[i])=="nan"):
        data.additional_fare[i] = 10.5

c:\users\isuru\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [499]:
data.head()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,...,pickup_year,pickup_month,pickup_date,pickup_hour,pickup_minute,drop_year,drop_month,drop_date,drop_hour,drop_minute
0,189123628,10.5,834.0,56.0,0.00000,64.0,6.86252,79.8993,6.90330,79.8783,...,2019,11,1,0,20,2019,11,1,0,34
1,189125358,10.5,791.0,47.0,0.00000,134.0,6.88589,79.8984,6.91373,79.8923,...,2019,11,1,0,56,2019,11,1,1,9
2,189125719,10.5,1087.0,80.0,0.00000,61.0,6.90839,79.8651,6.93669,79.9146,...,2019,11,1,1,8,2019,11,1,1,26
3,189127273,10.5,598.0,271.0,15.66380,68.0,6.92570,79.8895,6.92748,79.8971,...,2019,11,1,2,27,2019,11,1,2,37
4,189128020,10.5,1020.0,88.4,0.51304,NaN,6.87441,79.8615,6.84478,79.9290,...,2019,11,1,3,34,2019,11,1,3,51


In [500]:
lower_bound = 0.1
upper_bound = 0.95
res = data.additional_fare.quantile([lower_bound, upper_bound])
res

0.10    10.5
0.95    30.5
Name: additional_fare, dtype: float64

In [503]:
for i in range(0,len(data.additional_fare)):
    if(0>data.additional_fare[i]>250):
        data.additional_fare[i] =10.5

In [504]:
data.head()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,...,pickup_year,pickup_month,pickup_date,pickup_hour,pickup_minute,drop_year,drop_month,drop_date,drop_hour,drop_minute
0,189123628,10.5,834.0,56.0,0.00000,64.0,6.86252,79.8993,6.90330,79.8783,...,2019,11,1,0,20,2019,11,1,0,34
1,189125358,10.5,791.0,47.0,0.00000,134.0,6.88589,79.8984,6.91373,79.8923,...,2019,11,1,0,56,2019,11,1,1,9
2,189125719,10.5,1087.0,80.0,0.00000,61.0,6.90839,79.8651,6.93669,79.9146,...,2019,11,1,1,8,2019,11,1,1,26
3,189127273,10.5,598.0,271.0,15.66380,68.0,6.92570,79.8895,6.92748,79.8971,...,2019,11,1,2,27,2019,11,1,2,37
4,189128020,10.5,1020.0,88.4,0.51304,NaN,6.87441,79.8615,6.84478,79.9290,...,2019,11,1,3,34,2019,11,1,3,51


In [514]:
from sklearn.impute import SimpleImputer
data.fare

AttributeError: 'DataFrame' object has no attribute 'fare'

In [513]:
imputer = SimpleImputer(missing_values=np.nan,strategy='mean')
imputer = imputer.fit(data.fare)
imputed_data = imputer.transform(data.fare)
data[data.fare] = imputed_data

AttributeError: 'DataFrame' object has no attribute 'fare'

In [521]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17176 entries, 0 to 17175
Data columns (total 21 columns):
tripid                       17176 non-null int64
additional_fare              17176 non-null float64
duration                     17176 non-null float64
meter_waiting                17176 non-null float64
meter_waiting_fare           17176 non-null float64
meter_waiting_till_pickup    16974 non-null float64
pick_lat                     17176 non-null float64
pick_lon                     17176 non-null float64
drop_lat                     17176 non-null float64
drop_lon                     17176 non-null float64
label                        17176 non-null object
pickup_year                  17176 non-null int64
pickup_month                 17176 non-null int64
pickup_date                  17176 non-null int64
pickup_hour                  17176 non-null int64
pickup_minute                17176 non-null int64
drop_year                    17176 non-null int64
drop_month            